In [ ]:
import scipy.stats as stats
import numpy as np
import scipy.optimize as optimize
import math
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

log_plot = True

def calc_cs1_polynomial(true_model_coefficients, x):
    """
    Calculates the value of y for case study 1
    
    Parameters
    ----------
    true_model_coefficients: ndarray, The array containing the true values of Theta1 and Theta2
    x: ndarray, The list of xs that will be used to generate y
    
    Returns
    --------
    y_poly: ndarray, The noiseless values of y given theta_true and x
    """
    
    y_poly =  true_model_coefficients[0]*x + true_model_coefficients[1]*x**2 +x**3
       
    return y_poly

In [ ]:
%%time
# Create synthetic data assuming the following values for theta
Theta_Guess = np.array([1,-1])

##New Cell

# Evaluate model and add noise based on assumed theta values
# This generates experimental data points
Xexp = np.linspace(-2,2,5)
theta_ref = np.array([1.0, -1.0]) 
Yexp = calc_cs1_polynomial(theta_ref, Xexp)

##New Cell

# Evaluate model based on the assumed experimental values
X = np.linspace(np.min(Xexp),np.max(Xexp),100)
Y = Theta_Guess[0]*X + Theta_Guess[1]*X**2 + X**3

# Compare the experiments to the true model
plt.plot(X,Y,'b-',linewidth=2,label=r"$y$")
plt.plot(Xexp,Yexp,'r.',markersize=10,label=r"$y$")
plt.title("Plotting True Model and Synthetic Data")
plt.xlabel(r"$x$",fontsize=14)
plt.ylabel(r'$y$',fontsize=14)
plt.legend()
plt.show()

##New Cell

## define function that includes nonlinear model
def model(theta,x):
    '''
    Toy model
    Arguments:
        theta: parameter vector
        x: independent variable vector
    Returns:
        y_model: dependent variable prediction
    '''
    y_model = theta[0]*x + theta[1]*x**2 + x**3
    return y_model

print(model(Theta_Guess,Xexp))

##New Cell

# Create a function to optimize, in this case, least squares fitting
def regression_func(theta, x, y):
    '''
    Function to define regression function for least-squares fitting
    Arguments:
        theta: parameter vector
        x: independent variable vector
        y: dependent variable vector (measurements)
    Returns:
        e: residual vector
    '''
    
    error = y - model(theta,x); #NOTE: Least squares will calculate sse based off this to minimize
    
    return error
#New Cell

#Create a function to define the SSE for any Theta vector on a heat map.
def sse_func(xx, yy, x, y):
    '''
    Function to define define sum of squared error function for heat map
    Arguments:
        xx: An N X D array of all Theta1 values
            
        yy: An D X N array of all Theta2 values
        theta: parameter vector
        x: independent variable vector (predicted x values including noise)
        y: dependent variable vector (predicted y values on Heat Map)
    Returns:
        sse: N x N sum of squared error matrix of all generated combination of xx and yy
    '''
    sse = np.zeros([len(xx),len(yy)])
    
    for i in range(len(xx)):
        for j in range(len(yy)):
            theta = np.array([xx[i][j],yy[i][j]])
            sse[i][j] = sum((y - model(theta,x))**2) 
    
    return sse

In [ ]:
%%time
#New Cell

## specify initial guess
# sse_list = []
# for i in range(15):
theta0 = np.random.rand(2) * (2+2) -2
#     theta0 = Theta_Guess
#     print(theta0)

## specify bounds
# first array: lower bounds
# second array: upper bounds
bounds = ([-np.inf, -np.inf], [np.inf, np.inf])

## use least squares optimizer in scipy
# argument 1: function that takes theta as input, returns residual
# argument 2: initial guess for theta
# optional arguments 'bounds': bounds for theta
# optional arugment 'args': additional arguments to pass to residual function
# optional argument 'method': select the numerical method
#   if you want to consider bounds, choose 'trf'
#   if you do not want to consider bounds, try either 'lm' or 'trf'
Solution = optimize.least_squares(regression_func, theta0,bounds=bounds, method='trf',args=(Xexp, Yexp),verbose=2)

theta = Solution.x
print(Solution.fun)
print("theta = ",theta)
    
Y_pred2 = model(theta,Xexp)
error = regression_func(theta, Xexp, Yexp)
print(error)
SSE = np.sum(error**2)
print(SSE)

# sse_list = np.array(sse_list)
# print(np.median(sse_list))
# print(np.argmin(sse_list))

In [ ]:
#New Cell

# generate predictions
X_pred = np.linspace(np.min(Xexp),np.max(Xexp),20)
Y_pred = model(theta, X_pred)

Theta1_Map = np.linspace(-2,2,100)
Theta2_Map = np.linspace(-2,2,100)

x = Theta1_Map
y = Theta2_Map

# full coorindate arrays
xx, yy = np.meshgrid(x, y)
zz = sse_func(xx,yy,X_pred,Y_pred)

if log_plot == True:
    zz = np.log(zz)

plt.figure(figsize = (6,6))

cs = plt.contourf(xx, yy,zz, levels = 100, cmap = "autumn")

# plot color bar
# if np.amax(zz) < 1e-1:
#     cbar = plt.colorbar(cs, format='%.2e')
# else:
#     cbar = plt.colorbar(cs, format = '%2.2f')

# set font size in color bar
# cbar.ax.tick_params(labelsize=16)

# plot title in color bar
# cbar.ax.set_ylabel(r'$\mathbf{log(e(\theta))}$', fontsize=16, fontweight='bold')
#     print(p_GP_opt[0],p_GP_opt[1])



# Plot equipotential line
cs2 = plt.contour(cs, levels=cs.levels[::10], colors='k', alpha=0.7, linestyles='dashed', linewidths=3)

if np.amax(zz) < 1e-1:
    plt.clabel(cs2, fmt='%.2e', colors='k', fontsize=20)
else:
    plt.clabel(cs2, fmt='%2.2f', colors='k', fontsize=20)

plt.scatter(Theta_Guess[0],Theta_Guess[1], color="blue", s=100, label = "True Optimal Value", marker = (5,1))
plt.scatter(theta[0],theta[1], color="white",s=50, marker = ".",label = "NLR Optimal Value", edgecolor = "k", linewidth=0.3)
# plt.grid()

# plt.legend(fontsize=10,bbox_to_anchor=(0, 1.05, 1, 0.2),borderaxespad=0, loc = "lower left")
# plt.xlabel(r'$\mathbf{\theta_1}$',fontsize=20,fontweight='bold')
# plt.ylabel(r'$\mathbf{\theta_2}$',fontsize=20,fontweight='bold')
plt.xlim((np.amin(xx), np.amax(xx)))
plt.ylim((np.amin(yy),np.amax(yy))) 
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.locator_params(axis='y', nbins=5)
plt.locator_params(axis='x', nbins=5)
plt.minorticks_on() # turn on minor ticks
plt.tick_params(which="minor",direction="in",top=True, right=True)
plt.axis('square')
# plt.title("log(sse)", fontweight = "bold", fontsize = 24)

if log_plot == True:
    plt.title(r'$\mathbf{log(e(\theta))}$', weight='bold',fontsize = 24)
    plt.title("NLR", weight='bold',fontsize = 20)
#     plt.savefig("Figures/NLR_ln_SSE_poster.png", dpi=300, bbox_inches='tight')
else:
    plt.title('Non-Linear Regression SSE', weight='bold',fontsize = 16)
#     plt.savefig("Figures/NLR_SSE.png", dpi=300, bbox_inches='tight')
    

plt.show()
#New Cell

# create plot and compare predictions and experiments
plt.figure(figsize = (9,6))
plt.plot(Xexp,Yexp,'.g',markersize=20,label=r'$y$')
plt.plot(X,Y,'r-',linewidth=3,label=r'$f(\mathbf{\theta_{true}})$')
plt.plot(X_pred,Y_pred,'--b',linewidth=4,label=r'$f(\mathbf{\theta})$')
# plt.title("Predictions with $\\theta = [0.994,-1.00]$ vs Synthetic Data")
# plt.title("Predictions with $\\theta = [0.802,-0.757]$ vs Synthetic Data")
plt.legend(loc = "lower right", fontsize=30) #(bbox_to_anchor=(1.04, 1), borderaxespad=0
plt.xlabel(r'$x$',fontsize=30,fontweight='bold')
plt.ylabel(r'$y$',fontsize=30,fontweight='bold')

plt.locator_params(axis='y', nbins=5)
plt.locator_params(axis='x', nbins=5)
plt.minorticks_on() # turn on minor ticks
plt.tick_params(which="minor",direction="in",top=True, right=True)
# plt.grid(True)

# plt.savefig("Figures/sim_true_comp_poster.png", dpi=300, bbox_inches='tight')
plt.show()


##New Cell

#Plot error
Y_pred2 = model(theta,Xexp)
error = (Yexp - Y_pred2)
print("SSE = ", np.sum(error**2))
plt.plot(Y_pred2,error,"b.",markersize=20, label = "Error")
plt.title("Residuals")
plt.xlabel('Predicted Y')
plt.ylabel('Residuals vs. Predicted Value')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#Jacobian and Uncertainty Analysis
print("Jacobian =\n")
print(Solution.jac)
sigre = (error.T @ error)/(len(error) - 2)
Sigma_theta2 = sigre * np.linalg.inv(Solution.jac.T @ Solution.jac)
print("Covariance matrix:\n",Sigma_theta2)